In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1-1

In [ ]:
import os
import pandas as pd
from collections import defaultdict

# 이미지 데이터가 저장된 경로 설정
train_dir = "/content/drive/MyDrive/dataset/training_image"      # 실제 학습 데이터 디렉토리로 변경하세요
val_dir = "/content/drive/MyDrive/dataset/validation_image"      # 실제 검증 데이터 디렉토리로 변경하세요

# 성별 및 스타일별 이미지 수를 저장할 defaultdict 초기화
def count_images(directory):
    counts = defaultdict(lambda: defaultdict(int))
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            parts = filename.split('_')
            gender = "여성" if parts[-1].startswith('W') else "남성"
            style = parts[-2]
            counts[gender][style] += 1
    return counts

# Training 및 Validation 데이터셋에 대해 각각 카운트 수행
train_counts = count_images(train_dir)
val_counts = count_images(val_dir)

# defaultdict 데이터를 pandas DataFrame으로 변환
def dict_to_dataframe(counts):
    df = pd.DataFrame(counts).fillna(0).astype(int)
    df.index.name = '스타일'
    df.columns.name = '성별'
    return df

train_df = dict_to_dataframe(train_counts)
val_df = dict_to_dataframe(val_counts)

# 데이터를 원하는 형식으로 변환
def reshape_dataframe(df):
    melted_df = df.reset_index().melt(id_vars=['스타일'], var_name='성별', value_name='이미지 수')
    # 열 순서 변경
    melted_df = melted_df[['성별', '스타일', '이미지 수']]
    return melted_df

# Training 및 Validation 데이터 재구성
train_reshaped = reshape_dataframe(train_df)
val_reshaped = reshape_dataframe(val_df)

# 결과 출력
print("Training 데이터 통계")
print(train_reshaped)
print("\nValidation 데이터 통계")
print(val_reshaped)

Training 데이터 통계
    성별             스타일  이미지 수
0   여성        feminine    154
1   여성           space     37
2   여성        normcore    153
3   여성   bodyconscious     95
4   여성      genderless     77
5   여성          hiphop     48
6   여성          kitsch     91
7   여성        lingerie     55
8   여성        cityglam     67
9   여성        oriental     78
10  여성         minimal    139
11  여성          hippie     91
12  여성  sportivecasual    157
13  여성          lounge     45
14  여성         classic     77
15  여성      athleisure     67
16  여성         ecology     64
17  여성            punk     65
18  여성       powersuit    120
19  여성          popart     41
20  여성        military     33
21  여성           disco     37
22  여성          grunge     31
23  여성     metrosexual      0
24  여성            mods      0
25  여성            bold      0
26  여성             ivy      0
27  남성        feminine      0
28  남성           space      0
29  남성        normcore    364
30  남성   bodyconscious      0
31  남성      genderless  

1-2

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.utils import class_weight
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet18
from tqdm import tqdm
import numpy as np

# 학습 이미지 경로 및 생성 이미지 경로
train_dir = '/content/drive/MyDrive/dataset/training_image'
cropped_dir = '/content/drive/MyDrive/dataset/cropped_image'
val_dir = '/content/drive/MyDrive/dataset/validation_image'

# 업데이트된 평균과 표준편차로 데이터 전처리 및 변환 설정
transform_train1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=1.0),      # 좌우 반전 확률 100%
    transforms.RandomRotation(degrees=10),       # -10~10도 사이로 회전
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # 밝기, 대비, 채도, 색조 조정 (약하게)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4933, 0.4610, 0.4464], std=[0.2573, 0.2508, 0.2519])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4933, 0.4610, 0.4464], std=[0.2573, 0.2508, 0.2519])
])

In [ ]:
# 클래스 인덱스 매핑 생성 함수
# gender와 style 정보를 조합하여 고유한 클래스 인덱스를 생성
def create_class_to_idx(directories):
    # 고유한 (gender, style) 쌍을 저장하기 위한 집합
    classes = set()
    # 주어진 디렉토리 리스트에 대해 반복
    for directory in directories:
        for filename in os.listdir(directory):
            # 이미지 파일이 .jpg로 끝나는 경우만 처리
            if filename.endswith('.jpg'):
                parts = filename.split('_')  # 파일명에서 gender와 style 정보를 추출하기 위해 '_'로 분리
                # 파일명 끝에 'W'가 있는 경우를 여성으로 처리, 'M'이면 남성으로 처리
                gender = 0 if parts[-1].startswith('W') else 1  # 여성: 0, 남성: 1
                # 스타일 정보는 마지막에서 두 번째 부분에 위치
                style = parts[-2]
                # gender와 style의 조합을 classes 집합에 추가
                classes.add((gender, style))
    # (gender, style) 쌍에 고유한 인덱스를 부여하여 class_to_idx 딕셔너리 생성
    class_to_idx = {cls: idx for idx, cls in enumerate(sorted(classes))}
    return class_to_idx

# 이미지 데이터셋 클래스 정의
# 주어진 이미지 디렉토리에서 이미지와 라벨을 불러와서 메모리에 저장
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, class_to_idx, transform=None):
        # 이미지와 라벨을 저장할 리스트
        self.images = []
        self.labels = []
        self.class_to_idx = class_to_idx  # (gender, style) 조합과 인덱스 매핑 정보
        self.transform = transform  # 이미지에 적용할 전처리 변환

        # 주어진 디렉토리에서 파일을 하나씩 처리
        for filename in os.listdir(image_dir):
            if filename.endswith('.jpg'):
                parts = filename.split('_')  # 파일명을 '_'로 분리하여 파트별로 나눔
                # 파일명 끝에 'W'가 있는 경우를 여성으로, 'M'이면 남성으로 처리
                gender = 0 if parts[-1].startswith('W') else 1  # 여성: 0, 남성: 1
                # 스타일 정보는 마지막에서 두 번째 부분
                style = parts[-2]
                # (gender, style) 쌍을 cls 변수에 저장
                cls = (gender, style)
                # 클래스 인덱스 매핑에서 해당 (gender, style) 쌍에 해당하는 인덱스를 가져옴
                if cls in self.class_to_idx:
                    label = self.class_to_idx[cls]  # 고유한 클래스 인덱스

                # 유효한 라벨이 있는 경우만 처리
                if label is not None:
                    # 이미지 경로 생성
                    image_path = os.path.join(image_dir, filename)
                    # 이미지를 열어 RGB 형식으로 변환한 후 메모리에 로드
                    image = Image.open(image_path).convert('RGB')
                    # 전처리(transform)가 정의되어 있으면 적용
                    if self.transform:
                        image = self.transform(image)

                    # 메모리에 로드된 이미지를 images 리스트에 추가
                    self.images.append(image)
                    # 해당 이미지의 라벨을 labels 리스트에 추가
                    self.labels.append(label)

    def __len__(self):
        # 데이터셋의 전체 이미지 수 반환
        return len(self.labels)

    def __getitem__(self, idx):
        # 주어진 인덱스의 이미지와 라벨을 반환
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# 유효한 클래스 인덱스 매핑 생성
# training_image, cropped_image, validation_image 디렉토리의 모든 (gender, style) 조합에 대해 고유 인덱스를 생성
class_to_idx = create_class_to_idx([train_dir, cropped_dir, val_dir])

# 학습용 및 검증용 데이터셋 생성
# training_image 및 cropped_image에서 각각 데이터셋을 생성하고 전처리를 적용
original_train_dataset = CustomImageDataset(train_dir, class_to_idx, transform=transform_train1)
cropped_train_dataset = CustomImageDataset(cropped_dir, class_to_idx, transform=transform_train1)
# ConcatDataset을 사용하여 두 학습용 데이터셋을 하나로 결합
train_dataset = ConcatDataset([original_train_dataset, cropped_train_dataset])

# 검증용 데이터셋 생성 (전처리 변환을 transform_val로 설정)
val_dataset = CustomImageDataset(val_dir, class_to_idx, transform=transform_val)

In [ ]:
# 각 데이터셋의 고유 라벨 확인
print("Unique labels in original_train_dataset:", np.unique(original_train_dataset.labels))
print("Unique labels in cropped_train_dataset:", np.unique(cropped_train_dataset.labels))
print("Unique labels in val_dataset:", np.unique(val_dataset.labels))

Unique labels in original_train_dataset: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
Unique labels in cropped_train_dataset: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
Unique labels in val_dataset: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]


In [ ]:
# 로드 시 transform 적용할 수 있도록 새 클래스 정의
class TransformableLoadedImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.images = data['images']
        self.labels = data['labels']
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        #if self.transform:
        #    image = self.transform(image)  # 로드 시 transform 적용
        label = self.labels[idx]
        return image, label

# 저장된 데이터셋을 로드하고 transform 적용
original_train_data = torch.load('original_train_dataset_detect.pt')
cropped_train_data = torch.load('cropped_train_dataset_detect.pt')
val_data = torch.load('val_dataset_detect.pt')

# TransformableLoadedImageDataset 인스턴스 생성 (transform 적용)
original_train_dataset = TransformableLoadedImageDataset(original_train_data, transform=None)
cropped_train_dataset = TransformableLoadedImageDataset(cropped_train_data, transform=None)
val_dataset = TransformableLoadedImageDataset(val_data, transform=None)

train_dataset = ConcatDataset([original_train_dataset, cropped_train_dataset])

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=0, pin_memory=True)

# 데이터 확인 (선택적으로 추가)
print(f"Train Dataset size: {len(train_loader.dataset)}")
print(f"Validation Dataset size: {len(val_loader.dataset)}")

<ipython-input-16-44bf0e878f67>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  original_train_data = torch.load('original_train_dataset_detect.pt')
<ipython-input-16-44bf

Train Dataset size: 9731
Validation Dataset size: 951


In [ ]:
# ResNet-18 모델 정의 (pretrained=False)
model = resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 31)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 클래스 가중치 계산 및 손실 함수 정의
labels = np.concatenate([original_train_dataset.labels, cropped_train_dataset.labels])
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 옵티마이저와 학습률 스케줄러 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

print(f"Model is on device: {next(model.parameters()).device}")

# 학습 및 검증 함수
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct = 0.0, 0
    for inputs, labels in tqdm(loader, desc="Training"):
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

    return running_loss / len(loader.dataset), correct / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    running_loss, correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Validation"):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

    return running_loss / len(loader.dataset), correct / len(loader.dataset)

# 학습 및 검증 실행
num_epochs = 500
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    scheduler.step(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}]: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
          f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

# 모델 저장
torch.save(model.state_dict(), 'resnet18_fashion_classification_with_crops.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model is on device: cuda:0


Validation: 100%|██████████| 4/4 [00:00<00:00,  4.83it/s]


Epoch [1/500]: Train Loss=3.8254, Train Acc=0.0181, Val Loss=3.5012, Val Acc=0.0368


Validation: 100%|██████████| 4/4 [00:00<00:00,  4.70it/s]


...

Validation: 100%|██████████| 4/4 [00:00<00:00,  4.17it/s]


Epoch [31/500]: Train Loss=3.3820, Train Acc=0.0366, Val Loss=3.3921, Val Acc=0.0336


Validation: 100%|██████████| 4/4 [00:00<00:00,  4.51it/s]


Epoch [32/500]: Train Loss=3.3853, Train Acc=0.0311, Val Loss=3.4028, Val Acc=0.0189


Training:  74%|███████▍  | 29/39 [00:20<00:06,  1.44it/s]